In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice\\research'

In [4]:
os.chdir("../")

In [21]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    model_path:Path
    all_params:dict
    metrix_file_name:Path
    training_data:Path
    testing_data:Path
    #mlflow_uri: str   

In [16]:
from AutoMobilePriceRegression.constants import *
from AutoMobilePriceRegression.utils.common import read_yaml, create_directories,save_json,load_json

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params=self.params.GradientBoostingRegressor

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            all_params=params,
            metrix_file_name=Path(config.metrix_file_name),
            training_data=Path(config.training_data),
            testing_data=Path(config.testing_data),
            #mlflow_uri="https://dagshub.com/omars1234/Regression_Analysis.mlflow" 
        )

        return model_evaluation_config

In [18]:
import pickle
import pandas as pd
import numpy as np
from numpy import loadtxt

import urllib.request as request
from urllib.parse import urlparse
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error
#import mlflow
#import mlflow.sklearn
import time

In [19]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config= config

    def evaluation_metrics(self):
        with open("./artifacts/final_model.pkl","rb") as f:
           loaded_model=pickle.load(f)
        
        test_data=pd.read_csv('artifacts/prepare_base_model/test_data.csv', delimiter=',')
        x_test,y_test=(
            
            test_data.drop("price",axis=1),
            test_data["price"]
            )      
        rmse=np.sqrt(mean_squared_error(y_test,loaded_model.predict(x_test)))
        mse=mean_squared_error(y_test,loaded_model.predict(x_test))
        r2score=r2_score(y_test,loaded_model.predict(x_test))
        Scores={"rmse":rmse,"mse":mse,"r2score":r2score}
        save_json(path=Path("scores.json"),data=Scores)
        #return rmse,mse,r2score
    
"""
    def log_into_mlflow(self):

        with open("./artifacts/final_model.pkl","rb") as f:
           loaded_model=pickle.load(f)
        
        test_data=loadtxt('artifacts/prepare_base_model/test_data', delimiter=',')
        x_test,y_test=(
            
            test_data[:,:-1],
            test_data[:,-1]
            )
        

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted=loaded_model.predict(x_test)
            (rmse,mse,r2score)=self.evaluation_metrics(y_test,predicted)

            Scores={"rmse":rmse,"mse":mse,"r2score":r2score}
            save_json(path=Path("scores.json"),data=Scores)
            
        
            mlflow.log_metrics(
                {"rmse": rmse,"mse":mse ,"r2score": r2score}
                )
            mlflow.log_params(self.config.all_params)


            if tracking_url_type_store != "file":

                mlflow.sklearn.log_model(loaded_model, "model", registered_model_name="Price Regression")
            else:
                mlflow.sklearn.log_model(loaded_model, "model") 
"""              
        

'\n    def log_into_mlflow(self):\n\n        with open("./artifacts/final_model.pkl","rb") as f:\n           loaded_model=pickle.load(f)\n        \n        test_data=loadtxt(\'artifacts/prepare_base_model/test_data\', delimiter=\',\')\n        x_test,y_test=(\n            \n            test_data[:,:-1],\n            test_data[:,-1]\n            )\n        \n\n        mlflow.set_registry_uri(self.config.mlflow_uri)\n        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme\n        \n        with mlflow.start_run():\n            predicted=loaded_model.predict(x_test)\n            (rmse,mse,r2score)=self.evaluation_metrics(y_test,predicted)\n\n            Scores={"rmse":rmse,"mse":mse,"r2score":r2score}\n            save_json(path=Path("scores.json"),data=Scores)\n            \n        \n            mlflow.log_metrics(\n                {"rmse": rmse,"mse":mse ,"r2score": r2score}\n                )\n            mlflow.log_params(self.config.all_params)\n\n\n           

In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluation_metrics()
    #model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-14 14:28:06,216: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-14 14:28:06,220: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-14 14:28:06,222: INFO: common: created directory at: artifacts]
[2024-07-14 14:28:06,224: INFO: common: created directory at: artifacts/model_evaluation]
[2024-07-14 14:28:06,255: INFO: common: json file saved at: scores.json]
